# Running the CUES SUMMA setup

First, we have to install the setup, which simply fixes paths in the file manager.

In [1]:
! ./install_local_setup.sh

In [1]:
%pylab inline
import pysumma as ps
import xarray as xr

Populating the interactive namespace from numpy and matplotlib


<br>

## Instantiating a simulation object

To set up a `Simulation` object you must supply 2 pieces of information. 

First, the SUMMA executable; this could be either the compiled executable on your local machine, or a docker image. 
For this case, I'll assume that `summa.exe` is on your path. 
See the commented out `executable` for example of how you could also define the docker image. 
The string for the docker image simply came from looking at the output of the `docker images` command.

The second piece of information is the path to the file manager, which we just created through the install script.
To create the `Simulation` object you can then just pass these to the constructor as shown below.

In [2]:
executable = '/opt/local/bin/summa.exe'
#executable = 'docker.io/bartnijssen/summa:develop'
file_manager = './file_manager.txt'

s = ps.Simulation(executable, file_manager)

<br>

## Manipulating the configuration of the simulation object

Most of your interactions with pysumma will be facilitated through this `Simulation` object, so let's take some time to look through what is in it. 
What's contained in the `Simulation` object right after instantiation is generally just the input required for a SUMMA run.
For a more in depth discussion of what these are see the [SUMMA Input](https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/) page of the documentation.
There are several attributes of interest that can be examined. 
To see each of them you can simply `print` them. 
Here's a very high level overview of what's available:

* `s.manager` - the file manager
* `s.decisions` - the decisions file
* `s.output_control` - defines what variables to write out
* `s.force_file_list` - a listing of all of the forcing files to use
* `s.local_attributes` - describes GRU/HRU attributes (lat, lon, elevation, etc)
* `s.local_param_info` - listing of spatially constant local (HRU) parameter values
* `s.basin_param_info` - listing of spatially constant basin (GRU) parameter values
* `s.parameter_trial` - spatially distributed parameter values (will overwrite `local_param_info` values, can be either HRU or GRU)

Most of these objects have a similar interface defined, with exceptions being `local_attributes` and `parameter_trial`. Those two are standard `xarray` datasets. All others follow the simple API:

```
print(x)                 # Show the data as SUMMA reads it
x.get_option(NAME)       # Get an option
x.set_option(NAME, VALUE) # Change an option
x.remove_option(NAME)    # Remove an option
```

### Setting decisions

So, now that we've got a handle on what's available and what you can do with it, let's actually try some of this out. First let's just print out our decisions file so we can see what's in the defaults.


In [3]:
print(s.decisions)

simulStart    '2010-10-01 00:00'   ! simulation start time
simulFinsh    '2019-03-30 23:00'   ! simulation end time
soilCatTbl    ROSETTA              ! soil-category dataset
vegeParTbl    MODIFIED_IGBP_MODIS_NOAH ! vegetation-category dataset
soilStress    NoahType             ! choice of function for the soil moisture control on stomatal resistance
stomResist    BallBerry            ! choice of function for stomatal resistance
num_method    itertive             ! choice of numerical method
fDerivMeth    analytic             ! choice of method to calculate flux derivatives
LAI_method    monTable             ! choice of method to determine LAI and SAI
f_Richards    mixdform             ! form of Richards equation
groundwatr    qTopmodl             ! choice of groundwater parameterization
hc_profile    pow_prof             ! choice of hydraulic conductivity profile
bcUpprTdyn    nrg_flux             ! type of upper boundary condition for thermodynamics
bcLowrTdyn    zeroFlux            

<br> 

Great, we can see what's in there. But to be able to change anything we need to know the available options for each decision. Let's look at how to do that. For arbitrary reasons we will look at the `snowIncept` option, which describes the parameterization for snow interception in the canopy. First we will get it from the `decisions` object directly, and then query what it can be changed to, then finally change the value to something else.

<br>

In [4]:
# Get just the `snowIncept` option
print(s.decisions['snowIncept'])

# Look at what we can set it to
print(s.decisions['snowIncept'].available_options)

# Change the value 
s.decisions['snowIncept'] = 'stickySnow'
print(s.decisions['snowIncept'])

snowIncept    lightSnow            ! choice of parameterization for snow interception
['stickySnow', 'lightSnow']
snowIncept    stickySnow           ! choice of parameterization for snow interception


<br>

### Changing parameters

Much like the decisions we can manipulate the `local_param_info` file. First, let's look at what's contained in it

In [5]:
print(s.local_param_info)

upperBoundHead            |      -7.5d-1 |      -1.0d+2 |      -1.0d-2
lowerBoundHead            |      -1.0d+1 |      -1.0d+2 |      -1.0d-2
upperBoundTheta           |       0.2004 |       0.1020 |       0.3680
lowerBoundTheta           |       0.1100 |       0.1020 |       0.3680
upperBoundTemp            |     272.1600 |     270.1600 |     280.1600
lowerBoundTemp            |     274.1600 |     270.1600 |     280.1600
tempCritRain              |     273.1600 |     272.1600 |     274.1600
tempRangeTimestep         |       2.0000 |       0.5000 |       5.0000
frozenPrecipMultip        |       1.0000 |       0.5000 |       1.5000
snowfrz_scale             |      50.0000 |      10.0000 |    1000.0000
fixedThermalCond_snow     |       0.3500 |       0.1000 |       1.0000
albedoMax                 |       0.9000 |       0.2000 |       1.0000
albedoMinWinter           |       0.5500 |       0.6000 |       1.0000
albedoMinSpring           |       0.5500 |       0.3000 |       1.0000
albedo

<br>

<br>

Yikes, that's pretty long. Okay but we can change things. See:

In [6]:
# Print it
print(s.local_param_info['albedoMax'])

# Change the value
s.local_param_info['albedoMax'] = 0.9
print(s.local_param_info['albedoMax'])

# Change the value again - the right two values are currently unused by SUMMA though so this is superfluous
s.local_param_info['albedoMax'] = [0.9, 0.2, 1]
print(s.local_param_info['albedoMax'])

albedoMax                 |       0.9000 |       0.2000 |       1.0000
albedoMax                 |       0.9000 |       0.9000 |       0.9000
albedoMax                 |       0.9000 |       0.2000 |       1.0000


<br>

### Modifying output
And one more, we can also modify what get's written to output. 
The output control file represents the options available through columns of numeric values.
These numbers represent how to write the output. 
From the SUMMA documentation they are arranged as:

```
! varName          | outFreq | inst | sum | mean | var | min | max | mode
```

As before, let's look at what's in the `output_control` by simply printing it out.

<br>

In [7]:
print(s.output_control)

pptrate                              | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
airtemp                              | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarRainPlusMelt                   | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarSWE                            | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarThroughfallSnow                | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarThroughfallRain                | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarSnowSublimation                | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarInfiltration                   | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarExfiltration                   | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarSurfaceRunoff                  | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarSurfaceTemp                    | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarSenHeatTotal                   | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
scalarLatHeatTotal                   | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
averageInstantRunoff                 | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
averageRoutedRunoff               

<br>

Then, we can modify values in a couple of ways.

<br>

In [15]:
# Just change the frequency to daily output
print(s.output_control['scalarNetRadiation'])
print(s.output_control['scalarNetRadiation'].statistic)

# Change the output statistic from instantaneous to sum
s.output_control['scalarNetRadiation'] = [1, 0, 1, 0, 0, 0, 0, 0]
print(s.output_control['scalarNetRadiation'])
print(s.output_control['scalarNetRadiation'].statistic)

# We could also be more verbose:
s.output_control['scalarNetRadiation'] = {
    'period': 1, 'instant': 1, 'sum': 0, 
    'mean': 0, 'variance': 0, 'min': 0, 'max': 0
}
print(s.output_control['scalarNetRadiation'])
print(s.output_control['scalarNetRadiation'].statistic)

scalarNetRadiation                   | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
instant
scalarNetRadiation                   | 1 | 0 | 1 | 0 | 0 | 0 | 0 | 0
sum
scalarNetRadiation                   | 1 | 1 | 0 | 0 | 0 | 0 | 0 | 0
instant


<br>

## Running pysumma and manipulating output

Now that you've had an overview of how you can interact with SUMMA configurations through pysumma let's run a simulation. 
Before doing so we will reset our `Simulation` object, which will discard all of the changes we've made and load in a clean setup. 
Alternatively you could simply instantiate a new `Simulation` object.

In [10]:
#s.start('docker', run_suffix='_test')
s.reset()
s.start('local', run_suffix='_default')
s.monitor()
print(s._status)

Error


In [10]:
ds_default = s.output[0]

In [15]:
ds_default

<xarray.Dataset>
Dimensions:                   (hru: 1, time: 3102)
Coordinates:
  * time                      (time) datetime64[ns] 2010-10-02 ... 2019-03-30
  * hru                       (hru) int32 1
Data variables:
    scalarNetRadiation_total  (time, hru) float64 9.282e+06 ... 1.07e+07
Attributes:
    summaVersion:     v2.0.0
    buildTime:        Thu May  2 13:35:30 PDT 2019
    gitBranch:        cewa564_pangeo-0-gb4f5ec0
    gitHash:          b4f5ec0fa816116bd65736632b5e4e5e44f030bf
    soilStress:       NoahType
    stomResist:       BallBerry
    num_method:       itertive
    fDerivMeth:       analytic
    LAI_method:       monTable
    f_Richards:       mixdform
    groundwatr:       qTopmodl
    hc_profile:       pow_prof
    bcUpprTdyn:       nrg_flux
    bcLowrTdyn:       zeroFlux
    bcUpprSoiH:       liq_flux
    bcLowrSoiH:       zeroFlux
    veg_traits:       CM_QJRMS1988
    rootProfil:       powerLaw
    canopyEmis:       difTrans
    snowIncept:       lightSnow
   

In [ ]:
s = ps.Simulation(executable, file_manager)
s.decisions.snowLayers.set_value('CLM_2010')
s.local_param_info.zminLayer3.set_value([100., 0.05, 0.05])
s.local_param_info.zminLayer4.set_value([100., 0.1, 0.1])
s.local_param_info.zminLayer5.set_value([100., 0.25, 0.25])
s.local_param_info.zmaxLayer2_lower.set_value([1000., 0.2, 0.2])
s.local_param_info.zmaxLayer3_lower.set_value([1000., 0.5, 0.5])
s.local_param_info.zmaxLayer4_lower.set_value([1000., 1., 1.])
s.local_param_info.zmaxLayer2_upper.set_value([1000., 0.15, 0.15])
s.local_param_info.zmaxLayer3_upper.set_value([1000., 0.3, 0.3])
s.local_param_info.zmaxLayer4_upper.set_value([1000., 0.75, 0.75])

## Plotting and analysis

In [ ]:
print(s.stdout[-800:])

In [11]:
print(s.stderr)

NameError: name 's' is not defined

In [ ]:
ds = s.output